# Gerando Dados Sintéticos

Nesse notebook iremos realizar o processo de gerar dados sintéticos a partir de dados reais.
Iremos realizar esse processo pois é necesário realizar estatísticas em dados sigilosos, mas não podemos utilizar diretamente esses dados. Entra então a técnica de gerar dados sintéticos, onde iremos manter dados anonimos, mas conseguiremos fazer análizes estatísticas sem prejudicar a privacidade das pessoas.

In [2]:
!pip install sdv
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 3.5 MB/s 
     |████████████████████████████████| 53 kB 983 kB/s 
     |████████████████████████████████| 1.6 MB 40.1 MB/s 
     |████████████████████████████████| 131 kB 51.0 MB/s 
     |████████████████████████████████| 61 kB 193 kB/s 
     |████████████████████████████████| 47 kB 2.3 MB/s 
     |████████████████████████████████| 9.4 MB 49.6 MB/s 
     |████████████████████████████████| 295 kB 50.4 MB/s 
     |████████████████████████████████| 965 kB 37.6 MB/s 
     |████████████████████████████████| 662 kB 47.2 MB/s 
     |████████████████████████████████| 280 kB 10.7 MB/s 
     |████████████████████████████████| 15.3 MB 405 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: psutil
    Found existing installation: ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
#Importando dados
import pandas as pd
#Ignorando avisos
import warnings
warnings.filterwarnings('ignore')
#Importando modelos
from sdv.tabular import CTGAN
#Importando lib de avaliação dos modelos
from sdv.evaluation import evaluate
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.reports.utils import get_column_plot
from sdmetrics.single_column import BoundaryAdherence
from sdmetrics.single_column import StatisticSimilarity
#Importando lib de plotagem
import plotly.express as px
import plotly.graph_objects as go


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

dataset = pd.read_csv('C:/Users/rafae/OneDrive/Documents/GitHub/Assigment_2/dados/infringement_dataset_v2.csv')

In [3]:
display(dataset)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,first_name,last_name,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,Robert,Watkins,9251.775000,179055.000,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,Jane,Navarro,56553.990000,435436.500,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,David,Seagraves,5357.250000,24282.000,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,Deborah,Tandy,23651.175000,272203.260,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,David,Walker,12278.805000,150530.250,166638.75,6.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,Lawrence,Mcwayne,6605.910000,40455.000,40455.00,1.0,0.0,0.0,0.0,1.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,Belinda,Loveland,10074.465000,57595.500,56821.50,1.0,0.0,0.0,0.0,1.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,Aileen,Rasmussen,4770.405000,24162.750,20625.75,2.0,0.0,0.0,0.0,2.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,Mary,Marbury,10681.132500,121317.750,134439.75,2.0,0.0,0.0,0.0,2.0


## Tratamento de Dados

Aqui iremos fazer alguns tratamentos de dados.

Primeiro iremos retirar algumas colunas, pois não possuem um peso muito alto para a análise de dados e também diminui o custo computacional de gerar valores sintéticos, as colunas são as seguintes: 
   
    1. provided mobilephone - Não possui peso para a análise de dados.
    2. provided workphone - Não possui peso para a análise de dados.
    3. provided homephone - Não possui peso para a análise de dados.
    4. provided email - Não possui peso para a análise de dados.
    5. mobilephone reachable - Não possui peso para a análise de dados.
    6. first name - Não possui peso para a análise de dados.
    7. last name - Não possui peso para a análise de dados.

Após esse primeiro tratamento iremos selecionar algumas linhas randomicamente do dataset tratado. Podemos utilizar duas bibliotecas, sklearn ou pandas, para fazer essa seleção mas iremos utilizar pandas pois é mais simples de ser utlizada.

In [6]:
#Selecionando as colunas que serão retiradas do dataset.
dataset_tratado = dataset.drop(columns =  ['provided_mobilephone', 'provided_workphone', 
                                           'provided_homephone', 'provided_email', 'mobilephone_reachable', 
                                           'first_name', 'last_name'])
#Selecionamos 50% dos dados para gerar os dados sintéticos,
#utilizamos a função .sample, da lib pandas, para isso.
#Isso se dá para não deixarmos o processamentos desses dados pelo modelo que irá
#gerar os dados muito pesado.
dataset_tratado = dataset_tratado.sample(frac=0.15)
dataset_tratado.dropna(inplace=True)
display(dataset_tratado)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,num_req_bureau_qrt,num_req_bureau_year,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
183101,312222,0,Cash loans,F,Y,N,0,67500.0,634482.0,20596.5,...,2.0,2.0,7748.16750,29415.375000,83664.000000,2.0,1.0,0.0,1.0,4.0
200712,332660,0,Cash loans,M,Y,Y,0,225000.0,450000.0,27324.0,...,0.0,4.0,12374.08500,126844.714286,126537.428571,2.0,4.0,1.0,0.0,7.0
13683,115948,1,Revolving loans,F,Y,Y,0,220500.0,337500.0,16875.0,...,0.0,2.0,14862.69750,207968.625000,223767.562500,5.0,1.0,1.0,1.0,8.0
144694,267784,0,Cash loans,M,Y,Y,2,236700.0,755190.0,36459.0,...,1.0,6.0,11793.96500,145551.216000,150809.400000,9.0,6.0,0.0,0.0,15.0
186792,316547,0,Cash loans,F,Y,Y,0,202500.0,770292.0,35824.5,...,0.0,0.0,19009.80000,181107.562500,162484.312500,4.0,2.0,2.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20915,124372,0,Cash loans,F,Y,N,0,90000.0,568800.0,20430.0,...,1.0,1.0,34634.91000,367470.000000,376873.200000,2.0,2.0,1.0,0.0,5.0
21517,125065,0,Cash loans,M,Y,Y,0,202500.0,1125000.0,40540.5,...,1.0,0.0,11577.09000,134415.000000,144883.500000,3.0,0.0,0.0,0.0,3.0
195875,327120,0,Cash loans,M,Y,Y,1,225000.0,1467612.0,56029.5,...,1.0,1.0,6980.08500,59937.750000,55449.750000,6.0,0.0,0.0,0.0,6.0
277519,421561,0,Cash loans,F,Y,Y,2,90000.0,343800.0,16852.5,...,0.0,2.0,7661.09250,67500.000000,77573.250000,1.0,0.0,1.0,0.0,2.0


## Teste de Épocas e Discriminator

No código abaixo iremos rodas várias vezes o modelo com diferentes valores para epochs e discriminator_steps, assim iremos verificar qual se adequa melhor e qual é o melhor número dessas variáveis para obter o melhor score.

- Para epochs será terá uma faixa de 10 à 50, onde pulará de 5 em 5 passos.
- Para discriminator terá uma faixa de 1 à 5

Após finalizar os testes iremos guardar os resultados em um dataframe e assim iremos verificar qual se saiu melhor.

As informações dos modelos que serão rodados serão salvas na lista abaixo,será utilizada para gerarmos um dataset com as informações de:
- Quantidade de epochs
- Quantidade de discriminator_steps
- Overall_score

In [6]:
score  = []

Código criado para rodar várias vezes o modelo com quantidade de epochs e discriminator_steps diferentes, a quantidade são:
- Para a váriavel epochs: serão de 10 à 50 épocas, mas pulará 5 passos a cada iteração.
- Para a váriavel discriminator_steps: serão de 1 à 5.

Assim poderemos verificar qual o melhor conjunto de épocas e discriminator steps para se obter o melhor overall score.

In [ ]:
#AVISO! MODELO DEMORA EM VOLTA DE 6H30 PARA OBTER TODOS OS DADOS.
#OS RESULTADOS FORAM SALVOS EM UM CSV!

for epoch in range(5,30, 5):
    for discriminator in range(1,6):
        model = CTGAN(epochs=epoch, verbose=True, primary_key='loan_id', discriminator_steps=discriminator)
        model.fit(dataset_tratado)
        new_data = model.sample(num_rows=500)
        overall_score = evaluate(new_data, dataset_tratado)
        score.append([epoch,discriminator,overall_score])

Epoch 1, Loss G:  0.4804,Loss D: -0.5644
Epoch 2, Loss G:  0.8965,Loss D: -0.1071
Epoch 3, Loss G:  0.0141,Loss D:  0.1139
Epoch 4, Loss G:  0.1963,Loss D: -0.3263
Epoch 5, Loss G:  0.1257,Loss D: -0.1546
Epoch 6, Loss G: -0.2722,Loss D: -0.1404
Epoch 7, Loss G: -0.5298,Loss D:  0.0727
Epoch 8, Loss G: -0.7703,Loss D:  0.1855
Epoch 9, Loss G: -1.3285,Loss D:  0.0962
Epoch 10, Loss G: -1.3535,Loss D: -0.0842
Epoch 1, Loss G: -0.2324,Loss D: -1.0556
Epoch 2, Loss G: -0.5933,Loss D: -0.2713
Epoch 3, Loss G: -0.5743,Loss D: -0.3201
Epoch 4, Loss G: -1.3725,Loss D:  0.0307
Epoch 5, Loss G: -0.7915,Loss D: -0.5201
Epoch 6, Loss G: -1.3638,Loss D: -0.3509
Epoch 7, Loss G: -1.4234,Loss D: -0.1219
Epoch 8, Loss G: -1.8773,Loss D:  0.0600
Epoch 9, Loss G: -0.6229,Loss D: -0.1067
Epoch 10, Loss G: -1.2291,Loss D:  0.4009
Epoch 1, Loss G: -2.5075,Loss D: -1.4431
Epoch 2, Loss G: -1.3300,Loss D: -0.0456
Epoch 3, Loss G: -1.9357,Loss D: -0.6254
Epoch 4, Loss G: -0.2585,Loss D: -0.4688
Epoch 5, Loss 

Abaixo criamos um dataset com todos os resultados de cada iteração que foi realizada.

Conforme foi mencionado anteriormente os resultados foram salvos em csv, pois demora 6h30 para finalizar o processo de rodar o código acima.

In [7]:
#resultados = pd.DataFrame(score, columns = ['epocas', 'discriminator', 'overall_score'])
#resultados.to_csv('/content/drive/My Drive/Assignment 2/dados/resultados.csv')
resultados = pd.read_csv('C:/Users/rafae/OneDrive/Documents/GitHub/Assigment_2/dados/resultados.csv')
resultados.drop(columns=resultados.columns[0], axis=1, inplace=True)
display(resultados.head())

,epocas,discriminator,overall_score
0,10,1,0.906231
1,10,2,0.896955
2,10,3,0.910416
3,10,4,0.911469
4,10,5,0.904634


Abaxio selecionamos o Top 5 maiores Overall Score

In [8]:
top_5 = resultados.nlargest(5, 'overall_score').reset_index()
top_5.drop(columns=top_5.columns[0], axis=1, inplace=True)
display(top_5)

,epocas,discriminator,overall_score
0,15,3,0.915566
1,30,4,0.913500
2,10,4,0.911469
3,40,4,0.911305
4,10,3,0.910416


Abaixo vemos qual foi a combinação de épocas e discriminator steps que obtiveram o melhor overall score.
Vemos que mesmo o modelo tenha mais épocas o melhor overall socre veio com 15 épocas e 3 discriminator steps.


In [9]:
best_score = top_5[top_5['overall_score']==top_5['overall_score'].max()]
display(best_score)

,epocas,discriminator,overall_score
0,15,3,0.915566


Iremos utilizar a quantidade de épocas e discriminator steps com o melhor score no nosso modelo final.

In [11]:
epoca = int(best_score.iloc[0][0])
discriminator = int(best_score.iloc[0][1])

model = CTGAN(epochs=epoca, verbose=True, primary_key='loan_id', discriminator_steps=discriminator)
model.fit(dataset_tratado)

Epoch 1, Loss G: -1.5080,Loss D: -1.1985
Epoch 2, Loss G: -0.3425,Loss D: -0.4640
Epoch 3, Loss G: -0.7944,Loss D: -0.1760
Epoch 4, Loss G:  0.9811,Loss D: -0.5842
Epoch 5, Loss G:  1.3235,Loss D: -0.4728
Epoch 6, Loss G:  0.0324,Loss D: -0.0258
Epoch 7, Loss G:  1.1248,Loss D: -0.2683
Epoch 8, Loss G:  0.5146,Loss D: -0.7897
Epoch 9, Loss G: -0.4548,Loss D:  0.2802
Epoch 10, Loss G:  0.0931,Loss D:  0.2685
Epoch 11, Loss G:  0.7407,Loss D: -0.4330
Epoch 12, Loss G:  0.5519,Loss D: -0.1113
Epoch 13, Loss G:  0.1456,Loss D:  0.5983
Epoch 14, Loss G:  1.5163,Loss D: -0.6076
Epoch 15, Loss G:  0.7717,Loss D: -0.0306


Quando geramos um dataset com 500 linhas de dados sintéticos percebemos que o overall score e o data quality são altos, chegam por volta de 91%. Mas quando aumentamos o dataset para 15000 linhas de dados esses valores caem por volta de 2% nas mesmas varíaceis mencionadas anteriormente.

In [13]:
new_data = model.sample(num_rows=15000)
display(new_data)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,num_req_bureau_qrt,num_req_bureau_year,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,0,0,Cash loans,F,Y,Y,1,183424.0,920145.0,42381.2,...,0.0,8.0,19773.0,93102.0,10294.0,5.0,0.0,0.0,1.0,3.0
1,1,0,Cash loans,M,Y,Y,0,153868.0,754104.0,35085.0,...,0.0,4.0,12905.0,182621.0,74806.0,3.0,0.0,0.0,0.0,4.0
2,2,0,Revolving loans,M,Y,Y,1,201580.0,45000.0,7154.7,...,1.0,2.0,48810.0,20158.0,776568.0,3.0,0.0,0.0,0.0,5.0
3,3,0,Cash loans,M,Y,Y,1,148708.0,1194502.0,19634.9,...,0.0,0.0,17598.0,78545.0,88243.0,2.0,0.0,0.0,0.0,11.0
4,4,0,Cash loans,F,Y,N,1,139629.0,908666.0,49394.3,...,0.0,1.0,12511.0,263173.0,98535.0,2.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,0,Cash loans,F,Y,N,1,236638.0,741074.0,6002.8,...,0.0,0.0,7285.0,26622.0,164382.0,3.0,0.0,0.0,0.0,2.0
14996,14996,0,Cash loans,F,Y,Y,2,182269.0,565883.0,37746.5,...,0.0,2.0,8792.0,32880.0,127418.0,3.0,0.0,0.0,0.0,1.0
14997,14997,0,Cash loans,F,Y,Y,1,425170.0,589180.0,17757.3,...,0.0,2.0,8752.0,43386.0,84948.0,4.0,1.0,0.0,0.0,1.0
14998,14998,0,Cash loans,M,Y,Y,0,438795.0,570556.0,36445.4,...,0.0,0.0,84485.0,115609.0,198605.0,3.0,1.0,0.0,0.0,5.0


## Análise da qualidade dos dados sintéticos

Verificamos abaixo que o overall_score foi bastante alto: 91%!

Quando verificamos o KSComplement e o CSTest, percebemos que temos um score alto principalmente para o CSTest com 97%

In [14]:
overall_score = round(((evaluate(new_data, dataset_tratado))*100),2)
overall_score_sep = evaluate(new_data, dataset_tratado, aggregate=False)
print(f'Overall Score: {overall_score}% \n')
display(overall_score_sep)

Overall Score: 90.21% 



,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.832682,0.832682,0.0,1.0,MAXIMIZE,None
1,CSTest,Chi-Squared,0.971441,0.971441,0.0,1.0,MAXIMIZE,None


Vemos que a qualidade que foi capturada pelos dados sintéticos também tem um bom valor chegando em 89.67%

In [15]:
my_report = QualityReport()
my_report.generate(dataset_tratado, new_data, model.get_metadata().to_dict())

Creating report: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]



Overall Quality Score: 89.64%

Properties:
Column Shapes: 90.27%
Column Pair Trends: 89.0%


#### Data Quality
Abaixo vemos, por coluna, a qualidade dos dados que foram gerados pelo modelo, vemos que os dados que obtiveram esse valor menor foram referentes ao "score_ext" e isso impacta na análise de semelhança das estatísticas desses dados que faremos posteriormente.

In [16]:
my_report.get_visualization(property_name='Column Shapes')

Nos gráficos abaixo percebemos que os dados estão bastante parecidos, isso é bom para futuras análises com os dados sintéticos.

In [17]:
my_report.get_visualization(property_name='Column Pair Trends')

## Similaridade Estatísticas entre Dados Reais e Sintéticos (Média, Mediana e Desvio Padrão)

Aqui iremos selecionar apenas os dados numpericos para verificar a similaridade das estatísticas entre os dados reais e sintéticos.

In [18]:
dados_numericos = ['infringed','num_children', 'annual_income', 'credit_amount',
       'credit_annuity', 'goods_valuation', 'age', 'days_employed', 'car_age',
       'num_family_members', 'region_rating', 'score_ext_1', 'score_ext_2', 
       'score_ext_3','num_req_bureau_hour', 'num_req_bureau_day', 
       'num_req_bureau_week','num_req_bureau_month', 'num_req_bureau_qrt', 
       'num_req_bureau_year','past_avg_amount_annuity', 
       'past_avg_amt_application','past_avg_amt_credit', 'past_loans_approved',
       'past_loans_refused', 'past_loans_canceled', 'past_loans_unused',
        'past_loans_total']

In [19]:
similar_mean = []
similar_med = []
similar_std = []

#Média
for column in dados_numericos:
    column = str(column)
    resultados = StatisticSimilarity.compute(
        real_data=dataset_tratado[column],
        synthetic_data=new_data[column],
        statistic='mean')
    similar_mean.append([column,resultados])

#Mediana
for column in dados_numericos:
    column = str(column)
    resultados = StatisticSimilarity.compute(
        real_data=dataset_tratado[column],
        synthetic_data=new_data[column],
        statistic='median')
    similar_med.append([column,resultados])

#Desvio Padrão
for column in dados_numericos:
    column = str(column)
    resultados = StatisticSimilarity.compute(
        real_data=dataset_tratado[column],
        synthetic_data=new_data[column],
        statistic='std')
    similar_std.append([column,resultados])



similar_med = pd.DataFrame(similar_med, columns=['coluna', 'simi_mediana'])
similar_med['simi_mediana'] = similar_med['simi_mediana'].apply(lambda x: x*100).round(1)

similar_mean = pd.DataFrame(similar_mean, columns=['coluna', 'simi_media'])
similar_mean['simi_media'] = similar_mean['simi_media'].apply(lambda x: x*100).round(1)

similar_std = pd.DataFrame(similar_std, columns=['coluna', 'simi_std'])
similar_std['simi_std'] = similar_std['simi_std'].apply(lambda x: x*100).round(1)


Abaixo iremos verificar a similaridade estatística entre os dados numéricos entre os dados reais e os sitéticos. Se for 100 ou próximo de 100 mais parecidos as estatísticas dos sintéticos estão dos dados reais.

### Média

Verificamos que os dados possuem uma similaridade da média muito boa, mesmo nos dados que ficaram com o Quality Score baixos como nos dados "score_ext". 

In [20]:
fig_med = px.bar(similar_mean, x='coluna', y='simi_media', text_auto=True)
fig_med.update_layout(title_text='Similaridade das Médias(Dados Reais vs Dados Sintéticos)', 
                  xaxis={'categoryorder':'total descending'}, xaxis_tickangle=-90)
fig_med.show()

### Mediana
Quando olhamos para a mediana percebemos que os valores dos dados de "score_ext" ficam muito abaixo e com isso vemos o quanto o Quality Score afeta nos dados sintéticos

In [21]:
fig_med = px.bar(similar_med, x='coluna', y='simi_mediana', text_auto=True)
fig_med.update_layout(title_text='Similaridade das Medianas(Dados Reais vs Dados Sintéticos)', 
                  xaxis={'categoryorder':'total descending'}, xaxis_tickangle=-90)
fig_med.show()

### Standard Deviation

Verificamos que mesmo tendo uma média bastante similar entre todos os dados numéricos, a similaridade do desvio padrão é bem baixo entre os dados de "score_ext" porém já poderíamos prever esse comportamento pois verificamos que os dados sintéticos nessas colunas não ficaram com o Quality Score bons.

In [22]:
fig_std = px.bar(similar_std, x='coluna', y='simi_std', text_auto=True)
fig_std.update_layout(title_text='Similaridade das Standard Deviation(Dados Reais vs Dados Sintéticos)', 
                  xaxis={'categoryorder':'total descending'}, xaxis_tickangle=-90)
fig_std.show()

Quando olhamos os valores das colunas "score_ext_" percebemos o motivo de ter uma qualidade tão baixa, o modelo gera valores 1 ou 0, não valores entre esse intervalo pois estas colunas são previamente normalizadas e com isso explica a baixa qualidade desses dados.

In [32]:
import plotly.graph_objects as go

# Gerando gráfico boxplot para verificar a distribuição das colunas "score_ext_"
fig = go.Figure()
fig.add_trace(go.Box(y=new_data['score_ext_1']))
fig.add_trace(go.Box(y=new_data['score_ext_2']))
fig.add_trace(go.Box(y=new_data['score_ext_3']))
fig.show()

#Imprindo uma tabela que mostra as cinco primeiras linhas das colunas "score_ext_"
print('valores "score_ext_"')
display(new_data[['score_ext_1', 'score_ext_2', 'score_ext_3']].head())

#Imprindo valores unicos das colunas "score_ext_"
print(f"Valores unicos de 'score_ext_1': {new_data['score_ext_1'].unique()}")
print(f"Valores unicos de 'score_ext_2': {new_data['score_ext_2'].unique()}")
print(f"Valores unicos de 'score_ext_3': {new_data['score_ext_3'].unique()}")

valores "score_ext_"


,score_ext_1,score_ext_2,score_ext_3
0,1.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,1.0,1.0
3,0.0,1.0,0.0
4,0.0,0.0,1.0


Valores unicos de 'score_ext_1': [1. 0.]
Valores unicos de 'score_ext_2': [1. 0.]
Valores unicos de 'score_ext_3': [0. 1.]


## Importando dados sintéticos para análise

In [19]:
new_data.to_csv('C:/Users/rafae/OneDrive\Documents/GitHub/Assigment_2/dados/infringement_dataset_synthetic.csv')